<a href="https://colab.research.google.com/github/jodfedlet/TCC/blob/main/TCCII/Twitter/LDA_with_gensim_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Objetivos
 - Dividir o df por mês
 - Para cada subdf, realizar a experimentação:
    - definir numeros de tópicos
    - criar o modelo LDA
        - passar o número de tópico a cada vez
        - definir o valor de alfa e beta
    - Aplicar o modelo LDA para cada número de tópicos
    - Encontrar o número de tópicos que melhor representa os documentos (fase que pode ser realizada à partir de conhecimentos empíricos dos documentos ou por métrica de coerência)
 - Salvar o melhor número de tópicos para cada subdf   

In [10]:
!pip install --upgrade gensim
!pip install pyldavis
! pip install bs4
! pip install contractions
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 56.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=3f0218f96a740e15116e7150286c194498073878271b488c89b1ec4b614786e7
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/

In [160]:

import pandas as pd
import re
import json
import gzip
import gensim
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import pyLDAvis.gensim_models
# import pickle 
# import pyLDAvis

from google.colab import drive
drive.mount("/content/drive")

from pprint import pprint
from gensim import corpora
from bs4 import BeautifulSoup
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import datapath

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [248]:
try:
  dftweets = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/pre_processed_tweets.csv', lineterminator='\n', index_col=0)
except:
  print("Não foi possível carregar o arquivo")

In [ ]:
dftweets.head()

,datetime,tweet_id,user_id,text,count_word,text_preprocessed,bigrams_text,text_lematizados
1,2022-05-30 23:00:53+00:00,1531410348141592577,1473798922472767488,@BetoLuiz_RU @jonesmanoel_PCB @deccache O Hait...,15,haiti desgraca fato erros anulam acertos fala ...,haiti desgraca fato erros anulam acertos fala ...,haiti desgraca fato erro anular acerto falir s...
2,2022-05-30 22:57:20+00:00,1531409451122577410,1521912766151417856,O senhor vai investir na educação da Venezuela...,15,senhor vai investir educacao venezuela argenti...,senhor vai investir_educacao venezuela argenti...,senhor ir investir_educacao venezuela argentin...
3,2022-05-30 22:55:14+00:00,1531408925945434112,1253882714723409920,"@rsallesmma Que nada, na mão de vocês já tá vi...",7,nada mao voces virando mistura deserto saara h...,nada mao voces virando mistura deserto_saara h...,nado mao voces virar misturar deserto_saara haiti
4,2022-05-30 22:55:14+00:00,1531408923277807617,40320768,@JoppertLeonardo @jonesmanoel_PCB @deccache En...,21,entramos brics invadimos haiti mando bush lula...,entramos brics invadimos haiti mando_bush lula...,entrar brics invadir haiti mando_bush lula diz...
5,2022-05-30 22:37:49+00:00,1531404543568781313,1082697350441824257,"@alice_mader9 kkkkkk e era 2 haitiano, n enten...",3,haitiano entendi nada falavam,haitiano entendi_nada falavam,haitiano entendi_nada falar


In [249]:
dftweets.bigrams_text.head().tolist()

['haiti desgraca fato erros anulam acertos fala sobre guerra_ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir_educacao venezuela argentina haiti cuba etc paises_comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto_saara haiti',
 'entramos brics invadimos haiti mando_bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia_china inimigos mortais vai funcionar',
 'haitiano entendi_nada falavam']

In [250]:
dftweets['datetime'] = pd.to_datetime(dftweets.datetime, format='%Y-%m-%d')

###Agrupamento mensal

In [251]:
#agrupar df por mês e adicionar os períodos numa lista
mensal = [df_year_month for df_year_month in dftweets.groupby(pd.Grouper(key='datetime', freq='M', closed='left'))]

###Agrupamento trimestral

In [252]:
trimestral = [df_year_3month for df_year_3month in dftweets.groupby(pd.Grouper(key='datetime', freq='3M', closed='left'))]

###Definir como vai ser a repartição (mensal ou trimestral)

In [253]:
trim = True

In [254]:
df = trimestral if trim else mensal

In [255]:
len(df)

18

In [256]:
df_start_date = '01/01/2018'
df_end_date = '31/05/2022'

In [257]:
 #idenficar o período dos tweets
def get_tweets_period(idx, df):

  date_format = '%d/%m/%Y'
  if idx == 0:
    start_period = df_start_date
    end_period = df[idx][0].strftime(date_format)
  elif idx == len(df) - 1:
    current_start_period = df[idx-1][0].strftime(date_format)
    start_period = (datetime.datetime.strptime(current_start_period, date_format) + datetime.timedelta(days=1)).strftime(date_format)
    end_period = df_end_date
  else:
    current_start_period = df[idx-1][0].strftime(date_format)
    start_period = (datetime.datetime.strptime(current_start_period, date_format) + datetime.timedelta(days=1)).strftime(date_format)

    end_period = df[idx][0].strftime(date_format)
  return (start_period, end_period)   

###Listando os períodos

In [258]:
for idx, df_y_m in enumerate(df):
 start_period, end_period = get_tweets_period(idx, df)
 print(f'Perído de [{start_period} a {end_period}]')

Perído de [01/01/2018 a 31/03/2018]
Perído de [01/04/2018 a 30/06/2018]
Perído de [01/07/2018 a 30/09/2018]
Perído de [01/10/2018 a 31/12/2018]
Perído de [01/01/2019 a 31/03/2019]
Perído de [01/04/2019 a 30/06/2019]
Perído de [01/07/2019 a 30/09/2019]
Perído de [01/10/2019 a 31/12/2019]
Perído de [01/01/2020 a 31/03/2020]
Perído de [01/04/2020 a 30/06/2020]
Perído de [01/07/2020 a 30/09/2020]
Perído de [01/10/2020 a 31/12/2020]
Perído de [01/01/2021 a 31/03/2021]
Perído de [01/04/2021 a 30/06/2021]
Perído de [01/07/2021 a 30/09/2021]
Perído de [01/10/2021 a 31/12/2021]
Perído de [01/01/2022 a 31/03/2022]
Perído de [01/04/2022 a 31/05/2022]


###gensim oficial doc 
https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py

In [259]:
k_topicos = [3, 5, 7, 10, 15, 25]

In [261]:
no_above_dict = 0.5
dictionary_percent = 0.025
iterations = 1000
passes = 50
random_state = 50
   
#palavras que não são significativas
stop_words = ['nao','para','pra','aqui','trump', 'ir', 'ter', 'esse', 'todo','tudo', 'este', 'ser', 'querer', 'sao', 'pai', 'pro', 'ver', 'voce'] # 'fazer', 'dizer', 'falar'

###Procurar e salvar o melhor número de tópicos (k)

In [262]:
#best k topic of all
best_k_topic_of_each = []
lda_model = None
for idx, mini_df in enumerate(df):
  #df period and mini stats
  start_period, end_period = get_tweets_period(idx, df)
  print(f'Período de [{start_period} a {end_period}]')

  words_count = mini_df[1]['count_word'].sum()
  docs = mini_df[1].text_lematizados.tolist()
  tam_doc = len(docs)

  print(f'Tamanho do documento: {tam_doc} tweets, {words_count} palavras e {round(words_count/tam_doc, 2)} de médias de palavras')
  print('-'*80)
 
  texts = [[word for word in doc.split() if word not in stop_words and len(word) > 1] for doc in docs]
 
  #texts = docs_list[0:3]

  #criando o dicionário da representação dos documentos
  dictionary = corpora.Dictionary(texts)

  #Filtre as palavras que ocorrem em menos de 20 documentos ou em mais de 50% dos documentos.
  dictionary.filter_extremes(no_below=int(tam_doc * dictionary_percent), no_above=no_above_dict)

  #criando o corpus (bags-of-words dos documentos)
  corpus = [dictionary.doc2bow(text) for text in texts]

  # print(texts[0])
  # print(corpus[0])

  # print(f'Number of unique tokens: {len(dictionary)}')
  # print(f'Number of documents: {len(corpus)}')

  dic_best = {"c_v": 0,"k": k_topicos[0], 'top_topics': []}
 
  for i, k in enumerate(k_topicos):
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=k, chunksize=tam_doc, 
                         alpha='auto', eta='auto', iterations=iterations, random_state=random_state, passes=passes, eval_every=None)
    top_topics = lda_model.top_topics(corpus)

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_lda = round(coherence_model_lda.get_coherence(), 4)
    # print('Coherence Score: ', coherence_lda)
    # print(f'Para {k} tópicos -------------{coherence_lda}')

    if coherence_lda > dic_best["c_v"]:
      dic_best.update({'c_v': coherence_lda, 'k': k, 'top_topics': top_topics})
     

    # lda_model.save(f"/content/drive/My Drive/Colab Notebooks/TCC/lda_data_by_period/period_{period}_k_topic_{k}")
    # lda_model.save(f"/content/drive/My Drive/Colab Notebooks/TCC/lda_data_by_period/period_{period}_k_topic_{k}.bin")
    #pprint(top_topics)
   
    # for i, topic in enumerate(top_topics):
    #   topic_list = [word[1] for word in topic[0]]
    #   print(f'{i} : {topic_list}')
    
    # print('*'*200)
    # print()
  #adicionando o melhor k tópicos de cada corpus numa lista
  best_k_topic_of_each.append(dic_best["k"])  
  #break

print(best_k_topic_of_each)
best_of_all = max(best_k_topic_of_each,key=best_k_topic_of_each.count)
print(f'Best k solution is: {best_of_all}')

#salvando o melhor tópico
if trim:
   lda_model.save(f"/content/drive/My Drive/Colab Notebooks/TCC/k_topics_lda/{best_of_all}_trimestral")
   lda_model.save(f"/content/drive/My Drive/Colab Notebooks/TCC/k_topics_lda/{best_of_all}_trimestral.bin")
else: 
   lda_model.save(f"/content/drive/My Drive/Colab Notebooks/TCC/k_topics_lda/{best_of_all}")
   lda_model.save(f"/content/drive/My Drive/Colab Notebooks/TCC/k_topics_lda/{best_of_all}topic.bin")


Período de [01/01/2018 a 31/03/2018]
Tamanho do documento: 4796 tweets, 56126 palavras e 11.7 de médias de palavras
--------------------------------------------------------------------------------
Período de [01/04/2018 a 30/06/2018]
Tamanho do documento: 4443 tweets, 52714 palavras e 11.86 de médias de palavras
--------------------------------------------------------------------------------
Período de [01/07/2018 a 30/09/2018]
Tamanho do documento: 4574 tweets, 54364 palavras e 11.89 de médias de palavras
--------------------------------------------------------------------------------
Período de [01/10/2018 a 31/12/2018]
Tamanho do documento: 4349 tweets, 55752 palavras e 12.82 de médias de palavras
--------------------------------------------------------------------------------
Período de [01/01/2019 a 31/03/2019]
Tamanho do documento: 7341 tweets, 97960 palavras e 13.34 de médias de palavras
--------------------------------------------------------------------------------
Período de 

###Salvando os tópicos do melhor K tópicos de todos os períodos

In [332]:
k = 5 #atribuição manual conforme o resultado acima

In [286]:

try:
  if trim:
      LdaModel.load(f"/content/drive/My Drive/Colab Notebooks/TCC/k_topics_lda/{k}_trimestral.bin")
  else:
      LdaModel.load(f"/content/drive/My Drive/Colab Notebooks/TCC/k_topics_lda/{k}topic.bin")
      
  print(f'O melhor número de tópicos é: {k}')
  print()

  for idx, mini_df in enumerate(df):

    #df period and mini stats
    start_period, end_period = get_tweets_period(idx, df)
    print(f'Período de [{start_period} a {end_period}]')

    docs = mini_df[1].text_lematizados.tolist()
    tam_doc = len(docs)

    texts = [[word for word in doc.split() if word not in stop_words and len(word) > 1] for doc in docs]

    #criando o dicionário da representação dos documentos
    dictionary = corpora.Dictionary(texts)
    #Filtre as palavras que ocorrem em menos de 20 documentos ou em mais de 50% dos documentos.
    dictionary.filter_extremes(no_below=int(tam_doc * dictionary_percent), no_above=no_above_dict)
    #criando o corpus (bags-of-words dos documentos)
    corpus = [dictionary.doc2bow(text) for text in texts]

    newLdaModel = LdaModel(corpus=corpus, id2word=dictionary, num_topics=k, chunksize=tam_doc, 
                        alpha='auto', eta='auto', iterations=iterations, random_state=random_state, passes=passes, eval_every=None)
    
    period = str(start_period+'_'+end_period).replace('/','_')

    try:
      if trim:
        newLdaModel.save(f"/content/drive/My Drive/Colab Notebooks/TCC/lda_data_by_period/period_{period}_k_topic_{k}_trimestral")
        newLdaModel.save(f"/content/drive/My Drive/Colab Notebooks/TCC/lda_data_by_period/period_{period}_k_topic_{k}_trimestral.bin")
      else:
        newLdaModel.save(f"/content/drive/My Drive/Colab Notebooks/TCC/lda_data_by_period/period_{period}_k_topic_{k}")
        newLdaModel.save(f"/content/drive/My Drive/Colab Notebooks/TCC/lda_data_by_period/period_{period}_k_topic_{k}.bin")
    except Exception as e: 
      print(e) 
  
  print('Tópicos salvos com sucesso!')
except Exception as e: 
  print(e) 

O melhor número de tópicos é: 5

Período de [01/01/2018 a 31/03/2018]
Período de [01/04/2018 a 30/06/2018]
Período de [01/07/2018 a 30/09/2018]
Período de [01/10/2018 a 31/12/2018]
Período de [01/01/2019 a 31/03/2019]
Período de [01/04/2019 a 30/06/2019]
Período de [01/07/2019 a 30/09/2019]
Período de [01/10/2019 a 31/12/2019]
Período de [01/01/2020 a 31/03/2020]
Período de [01/04/2020 a 30/06/2020]
Período de [01/07/2020 a 30/09/2020]
Período de [01/10/2020 a 31/12/2020]
Período de [01/01/2021 a 31/03/2021]
Período de [01/04/2021 a 30/06/2021]
Período de [01/07/2021 a 30/09/2021]
Período de [01/10/2021 a 31/12/2021]
Período de [01/01/2022 a 31/03/2022]
Período de [01/04/2022 a 31/05/2022]
Tópicos salvos com sucesso!


###Mostrando os tópicos de cada período

In [309]:
invalid_print_chars = ['.','*','"',',']

In [331]:

try:
  # if trim:
  #   LdaModel.load(f"/content/drive/My Drive/Colab Notebooks/TCC/k_topics_lda/{k}_trimestral.bin")
  # else:
  #   LdaModel.load(f"/content/drive/My Drive/Colab Notebooks/TCC/k_topics_lda/{k}topic.bin")

  print(f'O melhor número de tópicos é: {k}')
  print()

  for idx, df_y_m in enumerate(df):
    start_period, end_period = get_tweets_period(idx, df)
    print(f'Período de [{start_period} a {end_period}]')
    print('-'*40)

    period = str(start_period+'_'+end_period).replace('/','_')

    if trim:
      LDA = LdaModel.load(f"/content/drive/My Drive/Colab Notebooks/TCC/lda_data_by_period/period_{period}_k_topic_{k}_trimestral.bin")
    else:  
      LDA = LdaModel.load(f"/content/drive/My Drive/Colab Notebooks/TCC/lda_data_by_period/period_{period}_k_topic_{k}.bin")

    topics = LDA.show_topics(num_words=20)

    #print(topics)
    for i, top in enumerate(topics):
      topic = (''.join([word.strip() for word in top[1] if not word.isdigit() and word not in invalid_print_chars])).split('+')
      print(f'{i} : {topic}')

    print('*'*220)
    print('\n\n')
except Exception as e:
  print(e)

O melhor número de tópicos é: 5

Período de [01/01/2018 a 31/03/2018]
----------------------------------------
0 : ['merda', 'paises', 'africa', 'presidente', 'eua', 'chamar', 'ainda', 'militar', 'achar', 'dizer', 'sobrar', 'brasil', 'falar', 'saber', 'caro', 'poder', 'agora', 'vir', 'rir', 'gente']
1 : ['haitiano', 'rir', 'ano', 'atar', 'agora', 'hoje', 'achar', 'vir', 'ainda', 'dizer', 'ficar', 'caro', 'dar', 'sobrar', 'brasil', 'falar', 'gente', 'chamar', 'saber', 'exercitar']
2 : ['dizer', 'poder', 'saber', 'caro', 'dar', 'vir', 'achar', 'africa', 'exercitar', 'ainda', 'falar', 'chamar', 'militar', 'brasileiro', 'rir', 'fazer', 'gente', 'ficar', 'agora', 'atar']
3 : ['fazer', 'sobrar', 'falar', 'gente', 'ainda', 'agora', 'vir', 'ficar', 'hoje', 'militar', 'rir', 'saber', 'haitiano', 'exercitar', 'africa', 'achar', 'ano', 'dizer', 'atar', 'caro']
4 : ['brasil', 'exercitar', 'ficar', 'onu', 'brasileiro', 'militar', 'ainda', 'agora', 'hoje', 'atar', 'ano', 'africa', 'chamar', 'rir', '